#### Importamos las librerías necesarias

In [1]:
import snscrape.modules.twitter as sntwitter
import pandas as pd
from sqlalchemy import create_engine
import pymysql

#### Webscrapping de los tweets

In [7]:
#Definir las fechas
start_date = '2022-11-21'
end_date = '2023-03-07'

In [8]:
query = '@TheBridge_Tech since:{} until:{}'.format(start_date, end_date)

In [33]:
#Función para hacer el scrapping de los tweets

def scrape_tweets(query):
    tweets = []
    for tweet in sntwitter.TwitterSearchScraper(query).get_items():
        if tweet.user.username != 'TheBridge_Tech':
            tweet_id = tweet.id
            text = tweet.rawContent
            date = tweet.date.strftime('%Y-%m-%d %H:%M:%S')
            author_id = tweet.user.id
            author_name = tweet.user.displayname
            author_username = tweet.user.username
            retweets = tweet.retweetCount
            replies = tweet.replyCount
            likes = tweet.likeCount
            quotes = tweet.quoteCount
            tweets.append([tweet_id, text, date, author_id, author_name, author_username, retweets, replies, likes, quotes])
    return tweets


In [34]:
tweets = scrape_tweets(query)

In [35]:
# Convertir la lista de tweets en un dataframe
tweets_df = pd.DataFrame(tweets, columns=['id', 'text', 'date', 'author_id', 'author_name', 'author_username', 'retweets', 'replies', 'likes', 'quotes'])


In [37]:
tweets_df.shape

(128, 10)

In [38]:
#Dataframe de los tweets
df_tweets = pd.DataFrame(columns=['id', 'text', 'date', 'author_id', 'retweets_count', 'replies_count', 'likes_count', 'quotes_count'])
df_tweets['id'] = tweets_df['id']
df_tweets['text'] = tweets_df['text']
df_tweets['date'] = tweets_df['date']
df_tweets['author_id'] = tweets_df['author_id']
df_tweets['retweets_count'] = tweets_df['retweets']
df_tweets['replies_count'] = tweets_df['replies']
df_tweets['likes_count'] = tweets_df['likes']
df_tweets['quotes_count'] = tweets_df['quotes']


In [39]:
df_tweets['author_id'].nunique()
#Hay 56 users únicos

56

In [40]:
#Dataframe de los users
df_users = tweets_df.loc[:, 'author_id':'author_username']

In [58]:
df_users.drop_duplicates(inplace=True)

In [59]:
df_users.shape

(56, 3)

#### Base de datos en AWS

In [43]:
#Credenciales AWS

username = "admin"
password = "pruebatecnica"
host = "technicaltest.cnvu29xw2umg.us-east-2.rds.amazonaws.com" 
port = 3306

In [44]:
#Conexión DB

db = pymysql.connect(host = host,
                     user = username,
                     password = password,
                     cursorclass = pymysql.cursors.DictCursor
)
cursor = db.cursor()

In [50]:
#Crear base de datos
create_db = '''CREATE DATABASE tweets_database'''
cursor.execute(create_db)

1

In [46]:
#Seleccionar base de datos
cursor.connection.commit()
use_db = ''' USE tweets_database'''
cursor.execute(use_db)

0

In [49]:
# Crear las tablas en la base de datos
cursor.execute('''
        CREATE TABLE tweets (
        id bigint PRIMARY KEY,
        text text,
        date timestamp,
        author_id bigint,
        retweets_count int,
        replies_count int,
        likes_count int,
        quotes_count int
        )
    ''')
cursor.execute('''
        CREATE TABLE users (
        author_id bigint PRIMARY KEY,
        author_name text,
        author_username text
        )
    ''')

0

In [50]:
# Crear sqlalchemy engine

engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}".format(user = username, pw = password, host = host, db = 'tweets_database'))
# engine = create_engine("mysql+pymysql://my_user:my_password@my_host/my_database")

In [51]:
# Cargar los dataframes en las tablas correspondientes
df_tweets.to_sql(name='tweets', con=engine, if_exists= 'append', index=False)
df_users.to_sql(name='users', con=engine, if_exists= 'append', index=False)

56

In [52]:
# Leer los datos para comprobar que se han ingestado correctamente
sql = '''SELECT * FROM tweets'''
cursor.execute(sql)
mi_tabla = cursor.fetchall()
mi_tabla

[{'id': 1594964278238654471,
  'text': 'Aprende hoy las profesiones del futuro, ¡inscríbete en el #BBKBootcamp ⏩ https://t.co/uJcCKObkPz\n\nJunto con @TheBridge_Tech podrás aprender sobre Desarrollo Web Full Stack, Data Science, Ciberseguridad y Diseño de producto digital UX UI. https://t.co/wdbmRphsAx',
  'date': datetime.datetime(2022, 11, 22, 8, 1, 31),
  'author_id': 702077405045899264,
  'retweets_count': 2,
  'replies_count': 0,
  'likes_count': 2,
  'quotes_count': 0},
 {'id': 1594965197613322241,
  'text': 'Ikasi gaur etorkizuneko lanbideak, izena eman #BBKBootcamp -ean ⏩ https://t.co/xvm6ekbcMQ\n\n@TheBridge_Tech -ekin batera,  Full Stack Web Garapena, Data Science, Zibersegurtasuna eta UX UI produktu digitalaren diseinuari buruz ikasteko aukera izango duzu. https://t.co/oxEOX35Osa',
  'date': datetime.datetime(2022, 11, 22, 8, 5, 10),
  'author_id': 702077405045899264,
  'retweets_count': 1,
  'replies_count': 0,
  'likes_count': 2,
  'quotes_count': 0},
 {'id': 1594968601265

In [53]:
db.commit()

In [28]:
# Cerrar la conexión a la base de datos
db.close()

In [54]:
df_tweets.to_csv('data/tweets.csv')
df_users.to_csv('data/users.csv')